In [ ]:
import os
import shutil
import glob
import tensorflow as tf
from sklearn.model_selection import train_test_split
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from tensorflow.keras.preprocessing.image import load_img, img_to_array, array_to_img

# Define paths
original_data_dir = 'E:\\FDS\\Project\\AIDER'  
train_path = "E:\\FDS\\Project\\train"
test_path = "E:\\FDS\\Project\\test"
TARGET_SIZE = (256, 256)  

# Split into Train and Test (with resizing)
os.makedirs(train_path, exist_ok=True)
os.makedirs(test_path, exist_ok=True)

for class_name in os.listdir(original_data_dir):
    class_path = os.path.join(original_data_dir, class_name)
    images = glob.glob(os.path.join(class_path, "*.jpg"))  

    train_images, test_images = train_test_split(images, test_size=0.2, random_state=42)

    os.makedirs(os.path.join(train_path, class_name), exist_ok=True)
    os.makedirs(os.path.join(test_path, class_name), exist_ok=True)

    for img_path in train_images:
        img = load_img(img_path)
        img = img.resize(TARGET_SIZE)  
        img.save(os.path.join(train_path, class_name, os.path.basename(img_path)))

    for img_path in test_images:
        img = load_img(img_path)
        img = img.resize(TARGET_SIZE)  
        img.save(os.path.join(test_path, class_name, os.path.basename(img_path)))



In [ ]:
import os
import glob
from tensorflow.keras.preprocessing.image import ImageDataGenerator, img_to_array, load_img

def augment_images(class_name, target_count):
    datagen = ImageDataGenerator(
        rotation_range=30, width_shift_range=0.2, height_shift_range=0.2, 
        shear_range=0.2, zoom_range=0.2, horizontal_flip=True, fill_mode="nearest"
    )

    class_train_path = os.path.join(train_path, class_name)
    images = glob.glob(os.path.join(class_train_path, "*.jpg"))

    current_count = len(images)
    if current_count >= target_count:
        print(f"Skipping {class_name}, already has {current_count} images.")
        return

    print(f"Augmenting {class_name} from {current_count} to {target_count} images...")

    generated = 0  
    images_per_original = max(1, (target_count - current_count) // current_count) 

    for img_path in images:
        if current_count + generated >= target_count:
            break 

        img = load_img(img_path)
        img = img.resize(TARGET_SIZE)  
        img = img_to_array(img)
        img = img.reshape((1,) + img.shape)

        save_prefix = os.path.basename(img_path).split(".")[0] + "_aug"
        i = 0
        for batch in datagen.flow(img, batch_size=1, save_to_dir=class_train_path, 
                                  save_prefix=save_prefix, save_format="jpg"):
            generated += 1
            i += 1
            if current_count + generated >= target_count or i >= images_per_original:
                break  

# Apply augmentation to all classes
for class_name in os.listdir(train_path):
    augment_images(class_name, 3200)

print("✅ Augmentation completed successfully!")


Augmenting collapsed_building from 408 to 3200 images...
Augmenting fire from 416 to 3200 images...
Augmenting flooded_areas from 420 to 3200 images...
Skipping normal, already has 3512 images.
Augmenting traffic_incident from 388 to 3200 images...
✅ Augmentation completed successfully!


In [ ]:
import tensorflow as tf
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Conv2D, MaxPooling2D, Flatten, Dense, Dropout

IMG_SIZE = (256, 256)
BATCH_SIZE = 32

# Load Train & Test Data using ImageDataGenerator
train_datagen = ImageDataGenerator(rescale=1.0/255)  
test_datagen = ImageDataGenerator(rescale=1.0/255)

train_gen = train_datagen.flow_from_directory(
    train_path,
    target_size=IMG_SIZE,
    batch_size=BATCH_SIZE,
    class_mode='categorical'  
)

val_gen = test_datagen.flow_from_directory(
    test_path,
    target_size=IMG_SIZE,
    batch_size=BATCH_SIZE,
    class_mode='categorical'
)

# Define CNN Model
model = Sequential([
    Conv2D(32, (3, 3), activation='relu', input_shape=(256, 256, 3)),
    MaxPooling2D(2, 2),

    Conv2D(64, (3, 3), activation='relu'),
    MaxPooling2D(2, 2),

    Conv2D(128, (3, 3), activation='relu'),
    MaxPooling2D(2, 2),

    Flatten(),
    Dense(128, activation='relu'),
    Dropout(0.5),
    Dense(5, activation='softmax')  
])

# Compile the Model
model.compile(optimizer='adam', loss='categorical_crossentropy', metrics=['accuracy'])

# Train the Model
history = model.fit(
    train_gen, 
    validation_data=val_gen,  
    epochs=10,  
    steps_per_epoch=len(train_gen),  
    validation_steps=len(val_gen)  
)

# Evaluate the Model
loss, accuracy = model.evaluate(val_gen)
print(f"Validation Accuracy: {accuracy * 100:.2f}%")

Found 15323 images belonging to 5 classes.
Found 1289 images belonging to 5 classes.


C:\Users\prave\AppData\Roaming\Python\Python312\site-packages\keras\src\layers\convolutional\base_conv.py:107: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(activity_regularizer=activity_regularizer, **kwargs)
C:\Users\prave\AppData\Roaming\Python\Python312\site-packages\keras\src\trainers\data_adapters\py_dataset_adapter.py:121: UserWarning: Your `PyDataset` class should call `super().__init__(**kwargs)` in its constructor. `**kwargs` can include `workers`, `use_multiprocessing`, `max_queue_size`. Do not pass these arguments to `fit()`, as they will be ignored.
  self._warn_if_super_not_called()


Epoch 1/10
479/479 ━━━━━━━━━━━━━━━━━━━━ 1187s 2s/step - accuracy: 0.4458 - loss: 1.3530 - val_accuracy: 0.6144 - val_loss: 1.0330
Epoch 2/10
479/479 ━━━━━━━━━━━━━━━━━━━━ 1139s 2s/step - accuracy: 0.7457 - loss: 0.6941 - val_accuracy: 0.7215 - val_loss: 0.7807
Epoch 4/10
479/479 ━━━━━━━━━━━━━━━━━━━━ 1113s 2s/step - accuracy: 0.7952 - loss: 0.5705 - val_accuracy: 0.8045 - val_loss: 0.5888
Epoch 5/10
479/479 ━━━━━━━━━━━━━━━━━━━━ 1205s 3s/step - accuracy: 0.8498 - loss: 0.4192 - val_accuracy: 0.7898 - val_loss: 0.6794
Epoch 6/10
479/479 ━━━━━━━━━━━━━━━━━━━━ 1149s 2s/step - accuracy: 0.9083 - loss: 0.2572 - val_accuracy: 0.8239 - val_loss: 0.6074
Epoch 8/10
479/479 ━━━━━━━━━━━━━━━━━━━━ 1137s 2s/step - accuracy: 0.9250 - loss: 0.2040 - val_accuracy: 0.8324 - val_loss: 0.6726
Epoch 9/10
479/479 ━━━━━━━━━━━━━━━━━━━━ 1301s 3s/step - accuracy: 0.9314 - loss: 0.1886 - val_accuracy: 0.8278 - val_loss: 0.7094
Epoch 10/10
479/479 ━━━━━━━━━━━━━━━━━━━━ 1380s 3s/step - accuracy: 0.9485 - loss: 0.1540 -